In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [12]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
# Create a parameter grid
param_grid = {
    'n_estimators': [100, 150],  # Example extension
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'min_weight_fraction_leaf': [0.0, 0.1],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 50],
    'min_impurity_decrease': [0.0, 0.1],
    'bootstrap': [True, False],
    'oob_score': [False],  # True can only be used if bootstrap=True
    'warm_start': [False, True],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.1],
    'max_samples': [None, 0.5]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 55296 candidates, totalling 276480 fits


C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
138240 fits failed out of a total of 276480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
46088 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constrain

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'ccp_alpha': [0.0, 0.1],
                         'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'max_leaf_nodes': [None, 10, 50],
                         'max_samples': [None, 0.5],
                         'min_impurity_decrease': [0.0, 0.1],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'min_weight_fraction_leaf': [0.0, 0.1],
                         'n_estimators': [100, 150], 'oob_score': [False],
                         'warm_start': [False, True]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [15]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'max_leaf_nodes': 10, 'max_samples': 0.5, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 150, 'oob_score': False, 'warm_start': False}: 0.950648360030511


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[55  3]
 [ 1 21]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9835423197492164

In [20]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [21]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [22]:
table=pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_ccp_alpha,param_class_weight,param_criterion,param_max_depth,param_max_features,...,param_warm_start,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.0,0.0,True,0.0,None,gini,None,auto,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
1,0.000000,0.000000,0.0,0.0,True,0.0,None,gini,None,auto,...,True,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
2,0.003125,0.006250,0.0,0.0,True,0.0,None,gini,None,auto,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
3,0.000000,0.000000,0.0,0.0,True,0.0,None,gini,None,auto,...,True,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
4,0.000000,0.000000,0.0,0.0,True,0.0,None,gini,None,auto,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55291,0.012390,0.021660,0.0,0.0,False,0.1,balanced,entropy,20,log2,...,True,"{'bootstrap': False, 'ccp_alpha': 0.1, 'class_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
55292,0.003336,0.003070,0.0,0.0,False,0.1,balanced,entropy,20,log2,...,False,"{'bootstrap': False, 'ccp_alpha': 0.1, 'class_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
55293,0.013414,0.010420,0.0,0.0,False,0.1,balanced,entropy,20,log2,...,True,"{'bootstrap': False, 'ccp_alpha': 0.1, 'class_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
55294,0.004705,0.002894,0.0,0.0,False,0.1,balanced,entropy,20,log2,...,False,"{'bootstrap': False, 'ccp_alpha': 0.1, 'class_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27649
